In [2]:
import os

In [3]:
%pwd

'c:\\Users\\sanoj\\end-to-end-ml-project-with-mlops\\research'

In [4]:
os.chdir("../")

In [5]:
os.environ["MLFLOW-TRACKING-URI"]="https://dagshub.com/Sanoj12/end-to-end-ml-project-with-mlops.mlflow"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_Path: Path
    model_path: Path
    all_params:dict
    metric_file_name: Path
    target_column:str
    mlflow_url : str
    

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)


        
        
        create_directories([self.config.artifacts_root])


        def get_model_evaluation_config(self) -> ModelEvaluationConfig:
            config = self.config.model_evaluation
            params = self.params.ElasticNet
            schema = self.schema.TARGET_COLUMN

            create_directories([config.root_dir])


            model_evaluation_config = ModelEvaluationConfig(
                root_dir=config.root_dir,
                test_data_Path=config.test_data_path,
                model_path= config.model_path,
                all_params=params,
                metric_file_name= config.metrics_file_name,
                target_column=schema.name,
                mlflow_url="https://dagshub.com/Sanoj12/end-to-end-ml-project-with-mlops" 
                
            )
            return model_evaluation_config


        

In [9]:
import pandas as pd
import os
import mlflow.sklearn
import mlflow
import numpy as np
import joblib
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from urllib.parse import urlparse


In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config



    def eval_metrics(self,actual ,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2_score = r2_score(actual,pred)

        return rmse, mae , r2_score     
    



    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_Path)
        model = joblib.load(self.config.model_path)


        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualitites = model.predict(test_x)
 
            rmse, mae, r2_score = self.eval_metrics(test_y, predicted_qualitites)

            scores = {"rmse":rmse, "mae":mae, "r2_score":r2_score} 

            save_json(path=Path(self.config.metric_file_name),data=scores)


            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2_score", r2_score)



In [11]:
###3pipelne

try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e


[2025-02-04 11:54:00,253 : INFO :common]: yaml file :config.yaml loaded successfully!]
[2025-02-04 11:54:00,259 : INFO :common]: yaml file :params.yaml loaded successfully!]
[2025-02-04 11:54:00,269 : INFO :common]: yaml file :schema.yaml loaded successfully!]


AttributeError: 'NoneType' object has no attribute 'artifacts_root'